https://blog.csdn.net/qq_41897800/article/details/114333091

torch里面的Transformer库要怎么用？例子如下：
https://github.com/CVxTz/time_series_forecasting/blob/26de501ec28bab1153a8a7abb2ad664d446ada57/time_series_forecasting/model.py#L69

看了一些实现，发现torch自带的transformer encoder decoder是没有pad，mask这样的功能的，得我们自己加进去。tnnd，这不误事嘛。

In [1]:
import numpy as np      # tang.npz的压缩格式处理
import os       # 打开文件
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchnet import meter
import tqdm

In [2]:
def get_data():
    if os.path.exists(data_path):
        datas = np.load(data_path, allow_pickle=True)      # 加载数据
        data = datas['data']  # numpy.ndarray
        word2ix = datas['word2ix'].item()   # dic
        ix2word = datas['ix2word'].item()  # dic
        return data, word2ix, ix2word

In [3]:
data_path = '../originalDataset/tang.npz'
data, word2ix, ix2word = get_data()
data = torch.from_numpy(data)
dataloader = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱

In [4]:
## 看看总共循环多少次
for dt in tqdm.tqdm(dataloader):
    print(dt.shape)
    print(dt.transpose(1, 0).shape)
    break

  0%|                                                  | 0/5758 [00:01<?, ?it/s]

torch.Size([10, 125])
torch.Size([125, 10])


In [5]:
data.shape

torch.Size([57580, 125])

In [15]:
def print_origin_poem(i):
    '''
    data里面每一行就是一首诗，总共57580首。
    本函数可以打印出data[i]对应的诗。
    '''
    poem_txt = [ix2word[int(i)] for i in data[i]]
    print("".join(poem_txt[poem_txt.index("<START>") + 1:-1]))
    print()
    print("".join(poem_txt[:]))
print_origin_poem(45423)

缓步出居处，过原边雁行。夕阳投草木，远水迎苍茫。野寺同蟾宿，云溪劚药尝。萧条霜景暮，极目尽堪伤。

</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>缓步出居处，过原边雁行。夕阳投草木，远水迎苍茫。野寺同蟾宿，云溪劚药尝。萧条霜景暮，极目尽堪伤。<EOP>


In [7]:
def generate(model, start_words, ix2word, word2ix):     # 给定几个词，根据这几个词生成一首完整的诗歌
    txt = []
    for word in start_words: # 机器学习
        txt.append(word) ## txt: [机,器,学,习]
    input = Variable(torch.Tensor([word2ix['<START>']]).view(1,1).long())      # tensor([8291.]) → tensor([[8291.]]) → tensor([[8291]])
    hidden = None
    num = len(txt)
    # print(input)
    for i in range(48):      # 最大生成长度
        output, hidden = model(input, hidden)
        if i < num:
            w = txt[i]
            input = Variable(input.data.new([word2ix[w]])).view(1, 1)
        else:
            top_index = output.data[0].topk(1)[1][0]
            w = ix2word[top_index.item()]
            txt.append(w)
            input = Variable(input.data.new([top_index])).view(1, 1)
        if w == '<EOP>':
            break
    return ''.join(txt)

def gen_acrostic(model, start_words, ix2word, word2ix):
    result = []
    txt = []
    for word in start_words:
        txt.append(word)
    input = Variable(
        torch.Tensor([word2ix['<START>']]).view(1, 1).long())  # tensor([8291.]) → tensor([[8291.]]) → tensor([[8291]])
    input = input.cuda()
    hidden = None

    num = len(txt)
    index = 0
    pre_word = '<START>'
    for i in range(48):
        output, hidden = model(input, hidden)
        top_index = output.data[0].topk(1)[1][0]
        w = ix2word[top_index.item()]

        if (pre_word in {'。', '!', '<START>'}):
            if index == num:
                break
            else:
                w = txt[index]
                index += 1
                input = Variable(input.data.new([word2ix[w]])).view(1,1)
        else:
            input = Variable(input.data.new([word2ix[w]])).view(1,1)
        result.append(w)
        pre_word = w
    return ''.join(result)

In [8]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers=2, batch_first=False)
        # lstm输入为：seq, batch, input_size
        # lstm输出为：seq * batch * 256; (2 * batch * 256,...)
        self.linear1 = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        seq_len, batch_size = input.size()
        if hidden is None:
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            h_0, c_0 = Variable(h_0), Variable(c_0)
        else:
            h_0, c_0 = hidden
        embeds = self.embeddings(input)     # (seq_len, batch_size, embedding_dim), (124,128,128) ## 难道不应该是(124, 10, 128)
        output, hidden = self.lstm(embeds, (h_0, c_0))      #(seq_len, batch_size, hidden_dim), (124,128,256)
        print(output.shape)
        output = self.linear1(output.view(seq_len*batch_size, -1))      # ((seq_len * batch_size),hidden_dim), (15872,256) → (15872,8293)
        return output, hidden


In [9]:
class Trsfmr_decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, nheads = 8, nlayers=6):
        super(Trsfmr_decoder,self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nheads) # , dim_feedforward=hidden_dim
        self.transformer_encoder = nn.TransformerEncoder(self.encode_layer, num_layers=nlayers)
        
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=nheads)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=nlayers)
        
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
    
    def forward(self, input, hidden = None):
        seq_len, batch_size = input.size()
        embeds_ = self.embeddings(input)
        embeds = self.embeddings(input)
        memory = self.transformer_encoder(embeds_)
        output = self.transformer_decoder(embeds, memory)
        # print(output.shape)
        output = self.linear1(output.view(seq_len*batch_size, -1))
        return output, None

In [10]:
def train():
#     modle = Net(len(word2ix), 128, 256)     # 模型定义：vocab_size, embedding_dim, hidden_dim —— 8293 * 128 * 256
    modle = Trsfmr_decoder(len(word2ix), 128, 256)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modle.parameters(), lr=1e-3)  # 学习率1e-3
    loss_meter = meter.AverageValueMeter()

    period = []
    loss2 = []
    for epoch in range(8):     # 最大迭代次数为8
        loss_meter.reset()
        for i, data in tqdm.tqdm(enumerate(dataloader)):    # data: torch.Size([128, 125]), dtype=torch.int32
            data = data.long().transpose(0, 1).contiguous()      # long为默认tensor类型，并转置, [125, 10]
            input, target = Variable(data[:-1, :]), Variable(data[1:, :])
            optimizer.zero_grad()
#             output, _ = modle(input)
            output, _ = modle(input)
            loss = criterion(output, target.view(-1))       # torch.Size([15872, 8293]), torch.Size([15872])
            loss.backward()
            optimizer.step()

            loss_meter.add(loss.item())    # loss:tensor(3.3510, device='cuda:0', grad_fn=<NllLossBackward>)loss.data:tensor(3.0183, device='cuda:0')

            period.append(i + epoch * len(dataloader))
            loss2.append(loss_meter.value()[0])
            
            # print(data)
            if (1 + i) % 2 == 0:       # 每575个batch可视化一次
                print(str(i) +':' + generate(modle, '床前明月光', ix2word, word2ix))

        torch.save(modle.state_dict(), 'model_poet_2.pth')
        break ## 我们暂时只跑1个epoch
    plt.plot(period, loss2)
    plt.show()

In [11]:
# def test():
# #     modle = Net(len(word2ix), 128, 256)     # 模型定义：vocab_size, embedding_dim, hidden_dim —— 8293 * 128 * 256
#     modle = Trsfmr(len(word2ix), 128, 256)
#     modle.load_state_dict(torch.load('model_poet.pth'))
#     modle.eval()
#     # txt = generate(modle, '床前明月光', ix2word, word2ix)
#     # print(txt)
#     txt = gen_acrostic(modle, '机器学习', ix2word, word2ix)
#     print(txt)

In [12]:
if __name__ == '__main__':
    data_path = '../originalDataset/tang.npz'
    data, word2ix, ix2word = get_data()
    data = torch.from_numpy(data)
    dataloader = torch.utils.data.DataLoader(data, batch_size=10, shuffle=True, num_workers=1)  # shuffle=True随机打乱
    train()
    # test()

2it [00:05,  2.75s/it]

1:床前明月光</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


4it [00:10,  2.44s/it]

3:床前明月光</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


6it [00:14,  2.30s/it]

...

153:床前明月光</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


156it [05:47,  2.28s/it]

155:床前明月光</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


157it [05:51,  2.24s/it]
Traceback (most recent call last):
  File "/Users/minkexiu/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/Users/minkexiu/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/minkexiu/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/Users/minkexiu/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 